In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
info1 = json.loads(open('runs/1/info.json').read())
# info2 = json.loads(open('runs/2/info.json').read())

p = figure(title='Confusion Matrix', plot_width=600, plot_height=400)

xy = np.array(info1['prop_acc_uncertainty_std'])
p.line(xy[:, 0], xy[:, 1], legend='std', color='blue', line_width=3)

xy = np.array(info1['prop_acc_uncertainty_entropy'])
p.line(xy[:, 0], xy[:, 1], legend='entropy', color='red', line_width=3)

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [41]:
info1 = json.loads(open('runs/4/info.json').read())
info2 = json.loads(open('runs/5/info.json').read())

p = figure(title='Confusion Matrix', plot_width=600, plot_height=400)

xy = np.array(info1['prop_acc_uncertainty_std'])
p.line(xy[:, 0], xy[:, 1], legend='std', color='blue', line_width=3)

xy = np.array(info2['prop_acc_uncertainty_entropy'])
p.line(xy[:, 0], xy[:, 1], legend='entropy', color='red', line_width=3)

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)